In [ ]:
import numpy as np
import os
import pathlib
import tensorflow as tf

from metrics import dice_coef, dice_loss, jaccard_distance

In [ ]:
image_size = (512, 512)
path_imgs = 'jpeg'
out_folder = 'a'
pathlib.Path(out_folder).mkdir(parents=True, exist_ok=True)

list_images = sorted([file for file in pathlib.Path(path_imgs).rglob('*')])

os.path.exists(path_imgs), len(list_images)

In [ ]:
for d in ['mask_unet', 'w_pred_mask']:
    p = os.path.join(out_folder, d)
    print(p)
    pathlib.Path(p).mkdir(parents=True, exist_ok=True)

model = tf.keras.models.load_model('unet.h5', custom_objects = {'dice_loss': dice_loss, 'dice_coef': dice_coef, 'jaccard_distance': jaccard_distance })

for i, file in enumerate(list_images):
    print(i, file.resolve())
    image = tf.keras.preprocessing.image.load_img(file.resolve())
    image = tf.keras.preprocessing.image.img_to_array(image)
    image = image / 255
    image = image.reshape((1, 512, 512, 3))
    mask = model.predict(image)
    mask = mask[0, :, :, :]
    new_filename = os.path.join(out_folder, 'mask_unet', str(file.stem) + '.bmp')
    print(new_filename, mask.shape)
    tf.keras.preprocessing.image.save_img(new_filename, mask)

    mask = np.uint8(mask >= 0.5)
    image_segmented = image * mask
    image_segmented[image_segmented == 0] = 1
    image_segmented = image_segmented[0, :, :, :]
    image_segmented = tf.keras.preprocessing.image.array_to_img(image_segmented)
    new_filename = os.path.join(out_folder, 'w_pred_mask', str(file.stem) + '.png')
    tf.keras.preprocessing.image.save_img(new_filename, image_segmented)